## 4.pm Est 
## 1.00pm  copy snapshotid from us-east-1 to us-east-1

### Working with snapshots.

## listing all snaps 

In [49]:
import boto3
from botocore.exceptions import ClientError
import time
from pprint import pprint
from datetime import datetime
ec2 = boto3.session.Session(profile_name="shared").resource('ec2', region_name='us-east-1')
sts = boto3.session.Session(profile_name="shared").client('sts')
AccounId = sts.get_caller_identity()['Account']

In [37]:
# at 06:00:00
# getting account id
# 2022/07/03 


def list_snapshots():
    """This function list all snapshot taken at 5.05
    """
    today = datetime.now()
    start_time = str(datetime(today.year, today.month, today.day, 20,5,56))
    filter_snap_size = {'Name': 'volume-size', 'Values': ['40']}
    snapshots = ec2.snapshots.filter(Filters=[filter_snap_size],OwnerIds=[AccounId])
    for each_snaps in snapshots:
        if each_snaps.start_time.strftime('%Y-%m-%d %H:%M:%S') == start_time:
            print(each_snaps.snapshot_id, each_snaps.start_time.strftime('%Y-%m-%d %H:%M:%S'))
    
list_snapshots()  


snap-0c6a97f6bed20031a 2022-07-03 20:05:56


In [60]:

def copy_snapshot_from_source_target(target_region, account_id=None, source_rgion= None):
    """Copy snapshots from us-east-1, us-weast-1
    """
    # client object in the source region
    client = boto3.session.Session(profile_name="shared").client('ec2', region_name='us-east-1')
    
    if source_rgion is None:
        source_rgion = "us-east-1"
    if account_id is None:
        account_id = AccounId   
    response = client.describe_snapshots(OwnerIds=[account_id])['Snapshots']

    snapshot_id = [snap['SnapshotId'] for snap in response]
    
    # copy snapshot from us-east-1 to us-west-2
    target_client = boto3.session.Session(profile_name="shared").client('ec2', region_name=target_region)
    print(f"copying snapshots from {source_rgion} to {target_region}")
    for each_snap in snapshot_id:
        target_client.copy_snapshot(
            Description= 'For DR reason',
            SourceRegion= source_rgion,
            SourceSnapshotId= each_snap
        )
    print("dsnapshot copied successfully!!")    

target_region = 'us-west-2'    
copy_snapshot_from_source_target(target_region, None, None)    
    

copying snapshots from us-east-1 to us-west-2
dsnapshot copied successfully!!


snap-0234bef90cbfb1fc6
snap-03f9c8191761b4031
snap-0c6a97f6bed20031a


In [61]:
copy_snapshot_from_source_target("us-east-2", None, None)  

copying snapshots from us-east-1 to us-east-2
dsnapshot copied successfully!!
